In [1]:
import pandas as pd
import numpy as np

# Train Test Split

In [ ]:
X=df.drop('mh_disorder_current', axis=1)

In [ ]:
y=df['mh_disorder_current']

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=32)

NameError: name 'X' is not defined

# Scores Generator

In [ ]:
def score_generator():
    y_pred = model.predict(x_train)

    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)

    metrics_dict = {
        'Mean Absolute Error (MAE)': mae,
        'Mean Squared Error (MSE)': mse,
        'Root Mean Squared Error (RMSE)': rmse,
        'R-squared (R2)': r2
    }

    return metrics_dict


# Linear Model

In [ ]:
from sklearn.linear_model import LinearRegression